In [ ]:
# =========================
# CHRIST UNIVERSITY CHATBOT
# Engineered by GNA
# =========================

!pip install -q openai ipywidgets

from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Javascript
import random

# 🔐 API KEY
API_KEY = "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"

client = OpenAI(
    api_key=API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# 🔗 Official Christ University Links
LINKS = {
    "Admissions": "https://christuniversity.in/admission",
    "Hostels": "https://christuniversity.in/hostels",
    "Academics": "https://christuniversity.in/academics",
    "Campuses": "https://christuniversity.in/campuses",
    "Placements": "https://christuniversity.in/placements",
    "Contact": "https://christuniversity.in/contact-us"
}

# 💡 Suggestive Question Pool
SUGGESTIONS_POOL = [
    "What is the admission process at Christ University?",
    "What courses are offered at Christ University?",
    "How are hostel facilities at Christ University?",
    "What is the eligibility for BCA at Christ University?",
    "How are placements at Christ University?",
    "Which Christ University campus is best for MBA?",
    "What entrance exams does Christ University conduct?",
    "What scholarships are available at Christ University?",
    "How is campus life and discipline at Christ University?",
    "How can I contact Christ University admissions?"
]

# 🎓 Header
header = widgets.HTML("""
<div style="background:linear-gradient(90deg,#0b5394,#073763);
            color:white;padding:20px;border-radius:16px;">
    <h2 style="margin:0;">🎓 Christ University Bot </h2>
    <p style="margin:2px 0 0;font-size:13px;opacity:0.9;">Engineered by GNA</p>
    <p style="margin:8px 0 0;font-size:14px;">
        Admissions • Academics • Hostels • Campuses • Placements • Student Life ,etc.
    </p>
</div>
""")

# 🧾 Output Area
output_area = widgets.Output(
    layout=widgets.Layout(height="360px", overflow_y="auto",
                          padding="14px", border="1px solid #dcdcdc",
                          background_color="#ffffff")
)

# ⌨️ Input Box
input_box = widgets.Text(
    placeholder="Enter your Christ University–related question here…",
    layout=widgets.Layout(width="100%", padding="10px")
)

# 🚀 Send Button
send_button = widgets.Button(
    description="Send",
    icon="paper-plane",
    button_style="primary",
    layout=widgets.Layout(width="25%", height="40px")
)

# 💡 Suggestions Container with black bg and white text
suggestion_container = widgets.VBox(
    layout=widgets.Layout(
        border='1px solid #000000',
        border_radius='12px',
        padding='10px',
        margin='10px 0px 20px 0px',
        background_color='#000000',
        width='100%'
    )
)

# ⚠️ Disclaimer
disclaimer = widgets.HTML("""
<p style="font-size:12px;color:#6b6b6b;text-align:center;">
⚠️ AI-generated responses may vary. Please verify important information on the official Christ University website.
</p>
""")

# 📦 UI Layout
chat_ui = widgets.VBox(
    [
        header,
        output_area,
        widgets.HTML("<b style='color:#ffffff; margin-bottom:8px;'>💡 Suggested Questions</b>"),
        suggestion_container,
        widgets.HBox([input_box, send_button], layout=widgets.Layout(gap="10px")),
        disclaimer
    ],
    layout=widgets.Layout(
        width="75%", margin="25px auto", padding="20px",
        border="1px solid d0d7de", border_radius="20px",
        background_color="#f9fbff", box_shadow="0px 10px 22px rgba(0,0,0,0.08)"
    )
)

display(chat_ui)

# 🧠 Chat Memory
bot_name = "CUB 🤖"
chat_history = []

# ⟲ Update Suggestion Buttons with white text on black bg
def update_suggestions():
    suggestion_container.children = []
    buttons = []
    for q in random.sample(SUGGESTIONS_POOL, 3):
        btn = widgets.Button(
            description=q,
            layout=widgets.Layout(width="100%", margin="6px 0"),
            style={"button_color": "#000000"},
            tooltip=q
        )
        btn.add_class("suggestion-btn")

        def on_click(b, question=q):
            input_box.value = question
            submit_message()

        btn.on_click(on_click)
        buttons.append(btn)

    suggestion_container.children = buttons

# CSS injection to style suggestion buttons white text on black background, with clean font
display(HTML("""
<style>
.suggestion-btn {
    background-color: #000000 !important;
    border: none !important;
}
.suggestion-btn .p-Button-label {
    color: white !important;
    font-family: Arial, sans-serif !important;
    font-weight: normal !important;
    font-size: 14px !important;
}
.suggestion-btn:hover {
    background-color: #222222 !important;
}
</style>
"""))

update_suggestions()

# 🤖 AI Response Generator
def generate_bot_response(user_input):
    system_prompt = (
        "You are a professional academic AI assistant specialized ONLY in Christ University.\n"
        "Provide detailed, accurate, and up-to-date responses.\n"
        "Use a formal, polished tone with clear structure.\n"
        "Correct spelling mistakes automatically.\n"
        "Politely refuse non–Christ University questions.\n\n"
        "Official Sources:\n" +
        "\n".join([f"{k}: {v}" for k, v in LINKS.items()]) + "\n\n"
        "Respond thoroughly and clearly, using the above official links when relevant."
    )

    messages = [{"role": "system", "content": system_prompt}]
    for chat in chat_history:
        messages.append({"role": "user", "content": chat["User"]})
        if chat["Bot"]:
            messages.append({"role": "assistant", "content": chat["Bot"]})
    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="openai/gpt-4o-mini",
        messages=messages,
        temperature=0.2,  # low temperature for factual, detailed answers
        max_tokens=800
    )

    return response.choices[0].message.content.strip()

# 📎 Submit Handler with improved scroll and thinking animation behavior
def submit_message(_=None):
    user_text = input_box.value.strip()
    if not user_text:
        return

    input_box.value = ""

    # Append the user message immediately to chat history for rendering and to keep scroll at bottom
    chat_history.append({"User": user_text, "Bot": None})

    with output_area:
        output_area.clear_output()
        # Render all chat including the new user message but no bot answer yet
        for chat in chat_history[:-1]:
            display(Markdown(f"**👤 You:** {chat['User']}"))
            display(Markdown(f"**🎓 {bot_name}:** {chat['Bot']}"))
            display(HTML("<hr style='margin:10px 0;'>"))
        # Show last user message only, bot answer pending
        last = chat_history[-1]
        display(Markdown(f"**👤 You:** {last['User']}"))

        # Thinking animation below last user message
        thinking_html = """
        <i style="color:#555;">
        🎓 CUB is analyzing your question…<br>
        📘 Generating a detailed response…<br>
        <img src="https://i.gifer.com/ZZ5H.gif" alt="Thinking..." width="40" style="margin-top:10px;" />
        </i>
        """
        thinking_display = display(HTML(thinking_html), display_id=True)

    # Scroll to bottom immediately so new user message + thinking animation visible
    display(Javascript('''
    (function scrollDown(){
        const outputDiv = document.querySelector('div.output_scroll');
        if(outputDiv){
            outputDiv.scrollTop = outputDiv.scrollHeight;
        }
    })()
    '''))

    # Generate bot reply (blocking)
    bot_reply = generate_bot_response(user_text)
    # Update the last entry with bot answer
    chat_history[-1]["Bot"] = bot_reply

    with output_area:
        output_area.clear_output()
        for chat in chat_history:
            display(Markdown(f"**👤 You:** {chat['User']}"))
            display(Markdown(f"**🎓 {bot_name}:** {chat['Bot']}"))
            display(HTML("<hr style='margin:10px 0;'>"))

    update_suggestions()

    # Scroll to bottom again to show the full updated chat with answer
    display(Javascript('''
    (function scrollDown(){
        const outputDiv = document.querySelector('div.output_scroll');
        if(outputDiv){
            outputDiv.scrollTop = outputDiv.scrollHeight;
        }
    })()
    '''))

# 🔘 Bind events
send_button.on_click(submit_message)
input_box.on_submit(submit_message)
